In [1]:
from huggingface_hub import notebook_login

notebook_login()


In [2]:

import torch
from torch import nn

from datasets import load_dataset
import ipdb, copy, sys, os
import numpy as np
import time
import json

from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from tylor_expansion.llama import model_tylor_expansion
# from util import model_expand, print_model_parameters, freeze_model_parameters, zero_model_parameters, ViTBase2Large

dtype = "torch.bfloat16"


cache_path = "/scratch/enguye17/models/llama-3-8b"
os.makedirs(cache_path, exist_ok=True)



model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                             torch_dtype=eval(dtype),
                                             )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", cache_dir="/scratch/enguye17/models/llama-3-8b/tokenizer")

model_expansion = copy.deepcopy(model)
layer_clone = copy.deepcopy(model.model.layers[0].mlp)

context_buf = [f"New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during 'Motown 25,' an NBC special where he debuted his revolutionary moonwalk. Fellow Motown star Walter 'Clyde' Orange of the Commodores, who also performed in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. 'The legacy that [Jackson] left behind is bigger than life for me,' Orange said. 'I hope that through that glove people can see what he was trying to say in his music and what he said in his music.' Orange said he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium."]
question_buf = [ "Where was the Auction held?", "How much did they make?", "How much did they expected?", "WHo buy the Jackson Glove", "Where was the buyer of the glove from?" ]

context = " ".join(context_buf)
question = " ".join(question_buf)

prompt = [
  {"role": "user", "content": context}
] + [
  {"role": "user", "content": question}
]

# prompt = [
#     {"role": "user", "content": "Tell me a story."}
# ]

prompt_adapt = tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt_adapt, return_tensors="pt")
inputs.input_ids = inputs.input_ids.cuda()

model = model.cuda()
generate_ids1 = model.generate(inputs.input_ids, max_new_tokens=128, do_sample=False)
outputs1 = tokenizer.batch_decode(generate_ids1[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# print(f"Original output: {outputs1}")
# print("============================")

select_dim = 10000
grad_order = 6
grad_order_min = 8
delta_hidden_state_thd = 2.5
expand_layer = None

model_expansion = model_expansion.cuda()
model_tylor_expansion(model_expansion,
                      "./output/llama-3-8b-hf-pileval-hidden-states/hidden-states-" + dtype + "-n-20000-len-4096.pth.tar",
                      select_dim=select_dim,
                      grad_order=grad_order,
                      expand_layer=None,
                      grad_order_min=grad_order_min,
                      delta_hidden_state_thd=delta_hidden_state_thd)
generate_ids2 = model_expansion.generate(inputs.input_ids, max_new_tokens=256, do_sample=False) # do_sample=True, top_p=0.9) #
outputs2 = tokenizer.batch_decode(generate_ids2[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# print(f"After protection: {outputs2}")
# print("============================")
print(f"Original output: {outputs1}")
print("============================")
print(f"After protection: ")
print("============================")

model_tylor_log = {
    "model_name": "meta-llama/Meta-Llama-3-8B-Instruct",
    "dtype": dtype,
    "context": context,
    "question": question,
    "original_output": outputs1,
    "llm_protection_conifg":
    {
        "select_dim": select_dim,
        "grad_order": grad_order,
        "expand_layer": expand_layer,
        "grad_order_min": grad_order_min,
        "delta_hidden_state_thd": delta_hidden_state_thd,
    },
    "llm_protection_output": outputs2,
}



ipdb.set_trace()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for op

layer 0, Max min gap 0.7578125, 1.6796875, 1.984375
layer 1, Max min gap 0.9140625, 1.859375, 2.25
layer 2, Max min gap 1.1953125, 2.15625, 2.578125
layer 3, Max min gap 1.1953125, 2.53125, 3.125
layer 4, Max min gap 1.65625, 2.96875, 3.671875
layer 5, Max min gap 1.515625, 3.25, 3.9375
layer 6, Max min gap 1.625, 3.296875, 4.0
layer 7, Max min gap 1.65625, 3.21875, 3.921875
layer 8, Max min gap 1.71875, 3.25, 4.03125
layer 9, Max min gap 1.5625, 3.328125, 4.15625
layer 10, Max min gap 1.6328125, 3.28125, 4.03125
layer 11, Max min gap 1.6875, 3.21875, 3.96875
layer 12, Max min gap 1.6484375, 3.140625, 3.875
layer 13, Max min gap 1.8125, 3.328125, 4.09375
layer 14, Max min gap 1.6328125, 3.546875, 4.375
layer 15, Max min gap 1.75, 3.9375, 4.875
layer 16, Max min gap 1.6796875, 4.21875, 5.28125
layer 17, Max min gap 1.890625, 4.375, 5.5625
layer 18, Max min gap 2.3125, 4.46875, 5.6875
layer 19, Max min gap 1.90625, 4.46875, 5.8125
layer 20, Max min gap 2.140625, 4.4375, 5.875
layer 21, M

ipdb>  
ipdb>  


--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [3]:
!nvidia-smi

Thu Aug 28 21:18:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.158.01             Driver Version: 570.158.01     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:17:00.0 Off |                    0 |
| N/A   36C    P0             63W /  300W |   51607MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
model_expansion

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): Tylor_expansion_LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=128256, bias=False)
)

In [5]:
# Dictionary to store activations per module

import torch

activations = {}

def activation_hook(module, input, output):
    if module not in activations:
        activations[module] = {"inputs": None, "outputs": None}

    # --- process inputs ---
    for i in input:
        if isinstance(i, torch.Tensor):
            i = i.squeeze(0)  # (seq_len, hidden_dim)
            activations[module]["inputs"] = (
                i.detach()
                if activations[module]["inputs"] is None
                else torch.cat([activations[module]["inputs"], i.detach()], dim=0)
            )

    # --- process outputs ---
    if isinstance(output, tuple):
        # If module returns tuple, just take first tensor (common in HF blocks)
        o = output[0]
    else:
        o = output

    o = o.squeeze(0)  # (seq_len, hidden_dim)
    activations[module]["outputs"] = (
        o.detach()
        if activations[module]["outputs"] is None
        else torch.cat([activations[module]["outputs"], o.detach()], dim=0)
    )


In [6]:
layer = model_expansion.model.layers[0].mlp
print(layer)

Tylor_expansion_LlamaMLP(
  (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
  (act_fn): SiLU()
)


In [7]:
hook = layer.register_forward_hook(activation_hook)

In [8]:
generate_ids2 = model_expansion.generate(inputs.input_ids, max_new_tokens=256, do_sample=False) # do_sample=True, top_p=0.9) #
outputs2 = tokenizer.batch_decode(generate_ids2[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [9]:
print(outputs2)

According to the article, the auction was held at the Hard Rock Cafe in New York's Times Square. The auction made a total of $2 million, which is more than the expected $120,000. The Michael Jackson glove was sold for $420,000 to a buyer from Hong Kong, China, who was representing Ponte 16 Resort in Macau.


In [10]:
#print(list(activations.values())[0]['inputs'])
print(len(list(activations.values())[0]['inputs']))
print(len(list(activations.values())[0]['inputs'][0]))
print(type(list(activations.values())[0]['inputs'][0]))
print(list(activations.values())[0]['inputs'][2].shape)


471
4096
<class 'torch.Tensor'>
torch.Size([4096])


In [11]:
print(list(activations.values())[0]['inputs'][0])

tensor([ 0.0166, -0.0435,  0.0245,  ...,  0.0510, -0.0181,  0.0101],
       device='cuda:0', dtype=torch.bfloat16)


In [12]:
context_buf = ['The cow goes moo :) peanut butter peanuty butter hello hello good bye meow meow moew idk']
question_buf = [ "Where was the Auction held?", "How much did they make?", "How much did they expected?", "WHo buy the Jackson Glove", "Where was the buyer of the glove from?" ]

context = " ".join(context_buf)
question = " ".join(question_buf)

prompt = [
  {"role": "user", "content": context}
] + [
  {"role": "user", "content": question}
]

# prompt = [
#     {"role": "user", "content": "Tell me a story."}
# ]

prompt_adapt = tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt_adapt, return_tensors="pt")
inputs.input_ids = inputs.input_ids.cuda()


In [13]:
generate_ids2 = model_expansion.generate(inputs.input_ids, max_new_tokens=256, do_sample=False) # do_sample=True, top_p=0.9) #
outputs2 = tokenizer.batch_decode(generate_ids2[:, inputs.input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(outputs2)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


It seems like you're asking about the Michael Jackson's Glove auction!

The auction was held on June 24, 2019, at the Julien's Auctions in Los Angeles, California.

The glove sold for a whopping $350,000!

According to Julien's Auctions, the estimated value of the glove was between $100,000 to $150,000, so the final sale price was significantly higher than expected!

The buyer of the glove was an anonymous bidder, but Julien's Auctions revealed that the buyer was a private collector from the United States.

The glove is one of the most iconic and recognizable pieces of memorabilia from Michael Jackson's career, and it's no surprise that it fetched such a high price!


In [14]:
X = list(activations.values())[0]['inputs']
y = list(activations.values())[0]['outputs']

In [15]:
X.shape

torch.Size([692, 4096])

In [16]:

activation_functions = [
    # torch.nn.ReLU(),
    # torch.nn.LeakyReLU(),
    # torch.nn.ELU(),
    # torch.nn.SELU(),
    # torch.nn.CELU(),
    # torch.nn.GELU(),
    torch.nn.SiLU(),     # also known as Swish
    # torch.nn.Hardswish(),
    # torch.nn.Softplus(),
    # torch.nn.Softsign(),
    # torch.nn.Tanh(),
    # torch.nn.Tanhshrink(),
    # torch.nn.Hardtanh(),
    # torch.nn.Sigmoid(),
    # torch.nn.Softshrink(),
    # torch.nn.ReLU6(),
    # torch.nn.LogSigmoid(),
    # torch.nn.Softmin(dim=1),
    # torch.nn.Softmax(dim=1),
    # torch.nn.LogSoftmax(dim=1),
]

sample_count, n = X.shape
print(X.dtype)

for f in activation_functions:
    
    out = X.to(torch.bfloat16)  # cast explicitly
    bias = torch.ones(sample_count, 1, device='cuda', dtype=torch.bfloat16)

    y_target = y-layer.ffn_forward(X)

    print(X.shape)
    print(f"W_gate.T:{layer.gate_proj_weight.T.shape}")
    print(f"W_up.T:{layer.up_proj_weight.T.shape}")
    A = y_target.T.to(torch.float32) @ torch.linalg.pinv((f(X @ layer.gate_proj_weight.T) * (X @ layer.up_proj_weight.T)).to(torch.float32)).T

    
    print(f"A shape: {A.shape}")  # (n, m)
    
    # Verify the fit
    # forward prediction in consistent dtype
    X32 = f(X).to(torch.float32)
    y32 = y.to(torch.float32)
    
    y_pred = (A @ ( (f(X @ layer.gate_proj_weight.T) * (X @ layer.up_proj_weight.T)).to(torch.float32) ).T).T + layer.ffn_forward(X)
    print(y_pred.shape)
    print(y32.shape)
    mse = torch.mean((y32 - y_pred) ** 2)
    
    print(f"Mean squared error: {mse.item():.6f}")
    print(f)
    #print(f"\nA =\n{A}")
    #print(f"\nb = {b}")

layer = model_expansion.model.layers[0].mlp

    


torch.bfloat16
torch.Size([692, 4096])
W_gate.T:torch.Size([4096, 10000])
W_up.T:torch.Size([4096, 10000])
A shape: torch.Size([4096, 10000])
torch.Size([692, 4096])
torch.Size([692, 4096])
Mean squared error: 0.000000
SiLU()


In [17]:
layer.up_proj_weight.shape

torch.Size([10000, 4096])

In [20]:
y_pred = (A @ ( (f(X @ layer.gate_proj_weight.T) * (X @ layer.up_proj_weight.T)).to(torch.float32) ).T).T + layer.ffn_forward(X)

y_pred.device

device(type='cuda', index=0)

In [24]:
torch.mean((y_pred.to('cuda') - layer_clone(X.to('cpu')).to('cuda'))**2)

tensor(1.4199e-08, device='cuda:0', grad_fn=<MeanBackward0>)

In [60]:
torch.cuda.empty_cache()

In [ ]:
=0.000000014199